### Para comenzar con el scrapping, importamos las funciones que tenemos en el archivo 'procesos'

In [1]:
from procesos import *

In [2]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import re

### Establecemos el nombre de las columnas que tendrá el dataframe

In [3]:
columnas = ['Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización']

### Llamamos a la función 'obtener_datos_obra' que realiza el proceso de web scraping (selenium), obtenemos los datos que queremos de la obra y lo convertimos en un dataframe

In [4]:
url = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/planta-potabilizadora-el-brillante/'
planta_el_brillante = obtener_datos_obra(url)

In [5]:
df = pd.DataFrame([planta_el_brillante]) #convertimos los datos en un dataframe

df = df.drop(df.columns[[2, 3]], axis=1) #eliminamos las columnas 2 y 3 que no tienen información

df['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df = df[[0,1,4,5,6,7,8,9,10,11,12,13,'Obra financiada por FONDEN','Monto comprometido USD (millones)',14,16,16,17]]
df.columns = columnas

### Vamos a realizar el mismo procedimiento para las demás obras que tenemos en nuestra página web que estamos scrapeando

In [6]:
url2 ='https://transparenciave.org/obrasinconclusas/obras_inconclusas/proyecto-tuy-iv-vialidad-tunel-de-desvio-estacion-de-bombeo-tuberia-construccion-de-la-presa-cuira-y-tendido-electrico/'
proyecto_tuy_iv = obtener_datos_obra(url2)

In [7]:
df2 = pd.DataFrame([proyecto_tuy_iv])
df2.columns = columnas

In [8]:
url3 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/sistema-de-riego-rio-tocuyo-aregue-las-huertas-la-otra-banda/'
sistema_riego = obtener_datos_obra(url3)

In [12]:
df3 = pd.DataFrame([sistema_riego])

df3 = df3.drop(df3.columns[[2, 3]], axis=1) 

df3['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df3['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df3 = df3[[0,1,4,5,6,7,8,9,10,11,12,13,'Obra financiada por FONDEN','Monto comprometido USD (millones)',14,16,16,17]]
df3.columns = columnas

In [17]:
url4 = 'https://transparenciave.org/obrasinconclusas/obras_inconclusas/sustitucion-de-tuberia-7-km-represa-los-quediches-de-48-pulgadas/'
tuberia = obtener_datos_obra(url4)

In [21]:
df4 = pd.DataFrame([tuberia])

df4['Obra financiada por FONDEN'] = 'S/I' #agregamos las columnas que hacen falta para mantener el formato del DF
df4['Monto comprometido USD (millones)'] = 'S/I'

#Reorganizamos el orden de las columnas
df4 = df4[[0,1,2,3,4,5,6,7,8,9,10,11,'Obra financiada por FONDEN','Monto comprometido USD (millones)',12,13,14,15]]
df4.columns = columnas

### Procedemos a juntar todos los dataframes

In [22]:
# Concatena los DataFrames en uno solo
data_agua = pd.concat([df,df2,df3,df4],ignore_index=True)

### Para conocer las coordenadas de cada una de las obras, hacemos uso de Geopy

In [24]:
#Asignamos como dirección el Municipio y Estado de cada obra para conocer sus coordenadas
municipio_estado = data_agua['Municipio'] + ', ' + data_agua['Estado']
lista_municipio_estado = municipio_estado.tolist()

In [25]:
latitudes = obtener_latitudes(lista_municipio_estado) #obtenemos la latitud de la obra

In [26]:
longitudes = obtener_longitudes(lista_municipio_estado) #obtenemos la longitud de la obra

In [27]:
data_agua['Latitud'] = latitudes #agregamos una nueva columna con todos los valores de las latitudes
data_agua['Longitud'] = longitudes #agregamos una nueva columna con todos los valores de las longitudes

### Extraemos la parte numérica de la columna 'Valor total de la obra' para tener un dato de tipo entero

In [33]:
# Aplicar la función a la columna 'Valor total de la obra'
data_agua['Valor total de la obra USD (millones)'] = data_agua['Valor total de la obra USD (millones)'].apply(extract_numeric_value)

data_agua['Valor total de la obra USD (millones)'] = data_agua['Valor total de la obra USD (millones)'].str.replace('.', '', regex=True).fillna('0').astype(int)

#Dividimos para tener el total de millones en un número mas legible
data_agua['Valor total de la obra USD (millones)'] = (data_agua['Valor total de la obra USD (millones)'] / 1000000).astype(int)

### Limpiamos la columna 'Monto comprometido', extraemos la parte numérica, llenamos los valores nulos con '0', transformamos a un tipo de dato float

In [35]:
data_agua['Monto comprometido USD (millones)'] = data_agua['Monto comprometido USD (millones)'].apply(extract_numeric_value)
data_agua['Monto comprometido USD (millones)'] = data_agua['Monto comprometido USD (millones)'].fillna('0')

# Convertir la columna a cadena, eliminar los puntos y reemplazar las comas por puntos
data_agua['Monto comprometido USD (millones)'] = data_agua['Monto comprometido USD (millones)'].astype(str)
data_agua['Monto comprometido USD (millones)'] = data_agua['Monto comprometido USD (millones)'].str.replace('.', '').str.replace(',', '.')

# Convertir la columna a tipo float
data_agua['Monto comprometido USD (millones)'] = pd.to_numeric(data_agua['Monto comprometido USD (millones)'], errors='coerce')

#Dividimos para tener el total de millones en un número más legible
data_agua['Monto comprometido USD (millones)'] = (data_agua['Monto comprometido USD (millones)'] / 1000000).astype(int)

### Cambiamos el tipo de dato de la columna 'Fecha estimada de inicio' por datetime y extraemos el año

In [36]:
# Convertir la columna 'Fecha estimada de inicio' a datetime
data_agua['Fecha estimada de inicio'] = pd.to_datetime(data_agua['Fecha estimada de inicio'], errors='coerce')
data_agua['Fecha estimada de inicio'] = data_agua['Fecha estimada de inicio'].dt.year

### Transformamos las columnas 'Ejecución física' y 'Ejecución financiera' en valores de tipo float, para ello llenamos los valores 'S/I' con 0 y extraemos la parte numérica

In [37]:
data_agua['Ejecución física %'] = data_agua['Ejecución física %'].apply(extract_numeric_value)
data_agua['Ejecución física %'] = data_agua['Ejecución física %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_agua['Ejecución física %'] = data_agua['Ejecución física %'].astype(float)

In [39]:
data_agua['Ejecución financiera %'] = data_agua['Ejecución financiera %'].apply(extract_numeric_value)
data_agua['Ejecución financiera %'] = data_agua['Ejecución financiera %'].str.replace(',', '.').replace({None: '0'})

# Convertir la columna a tipo float
data_agua['Ejecución financiera %'] = data_agua['Ejecución financiera %'].astype(float)

### Añadimos una nueva columna para indicar cuál es el sector correspondiente a la obra

In [40]:
data_agua['Sector'] = 'Agua'

In [41]:
#Reordenamos las columnas
nuevo_orden = ['Sector','Obra','Status','Ejecución física %','Ejecución financiera %','Estado','Municipio','Dirección','Personas afectadas','Ente responsable','Funcionario responsable','Empresa contratista','Valor total de la obra USD (millones)','Obra financiada por FONDEN','Monto comprometido USD (millones)','Fecha estimada de inicio','Fecha real de inicio','Fecha culminación estimada de la obra','Fecha de paralización', 'Latitud', 'Longitud']
data_agua = data_agua[nuevo_orden]

In [42]:
data_agua

,Sector,Obra,Status,Ejecución física %,Ejecución financiera %,Estado,Municipio,Dirección,Personas afectadas,Ente responsable,...,Empresa contratista,Valor total de la obra USD (millones),Obra financiada por FONDEN,Monto comprometido USD (millones),Fecha estimada de inicio,Fecha real de inicio,Fecha culminación estimada de la obra,Fecha de paralización,Latitud,Longitud
0,Agua,PLANTA POTABILIZADORA EL BRILLANTE,Paralizada,78.0,0.0,Estado Zulia,Guajira,Sector Guasare,Entre 5.001 y 100.000,Vicepresidencia Sectorial de Obras Públicas y ...,...,S/I,180,S/I,0,2016.0,Segundo trimestre del año 2016,Segundo trimestre del año 2016,2017,10.692780,-71.634994
1,Agua,"PROYECTO TUY IV (VIALIDAD, TÚNEL DE DESVIO, ES...",Paralizada,67.0,0.0,Estado Miranda,Acevedo,Pueblo de Panaquire,Más de 1.000.001,Hidrológica de la Región Capital C.A (HIDROCAP...,...,Camargo Correa,1016,Si,0,2009.0,S/I,2017,S/I,10.408194,-66.294991
2,Agua,"SISTEMA DE RIEGO RÍO TOCUYO, AREGUE, LAS HUERT...",Paralizada con daños en su infraestructura,0.0,50.0,Estado Lara,Torres,"Parroquia Camacaro y Chiquinquirá (Aregue), La...",Entre 0-5.000,HIDROLARA,...,Meolca C.A.,355,S/I,0,2002.0,S/I,S/I,2006-2007,10.189435,-69.893117
3,Agua,SUSTITUCIÓN DE TUBERÍA 7 KM REPRESA LOS QUEDIC...,Operativa con daños en su infraestructura,92.0,0.0,Estado Lara,Torres,Caserío Burere,Entre 100.001 y 1.000.000,Empresa Noroccidental de Mantenimiento y Obras...,...,S/I,13,S/I,0,NaN,Diciembre 2014,2015,2014 / Reinicia 2015,10.189435,-69.893117


### Exportamos nuestro dataframe

In [43]:
data_agua.to_csv('obras_agua.csv', index=False)